# Multi-period OPF with renewable example

Renewable generations are treated as negative load at each buses.

Renewable generations include current solar (NY-Sun), future solar (GIS), onshore wind (USWTDB), and offshore wind (NYSERDA, GIS).

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from nygrid.nygrid import NYGrid

In [2]:
# Set up directories
cwd = os.getcwd()
if 'examples' in cwd:
    parent_dir = os.path.dirname(cwd)
    data_dir = os.path.join(parent_dir, 'data')
else:
    data_dir = os.path.join(cwd, 'data')

grid_data_dir = os.path.join(data_dir, 'grid')
if not os.path.exists(grid_data_dir):
    raise FileNotFoundError('Grid data directory not found.')

print('Grid data directory: {}'.format(grid_data_dir))

fig_dir = os.path.join(os.path.dirname(data_dir), 'figures')
print('Figure directory: {}'.format(fig_dir))

results_dir = os.path.join(os.path.dirname(data_dir), 'results')
print('Results directory: {}'.format(results_dir))

solar_data_dir = os.path.join(data_dir, 'solar')
print('Solar data directory: {}'.format(solar_data_dir))

onshore_wind_data_dir = os.path.join(data_dir, 'onshore_wind')
print('Onshore wind data directory: {}'.format(onshore_wind_data_dir))

offshore_wind_data_dir = os.path.join(data_dir, 'offshore_wind')
print('Offshore wind data directory: {}'.format(offshore_wind_data_dir))

## Read grid data

### Read generator profiles

In [3]:
start_date = datetime(2018, 1, 1, 0, 0, 0)
end_date = datetime(2019, 1, 1, 0, 0, 0)
timestamp_list = pd.date_range(start_date, end_date, freq='1D')

# Read load profile
load_profile = pd.read_csv(os.path.join(grid_data_dir, f'load_profile_{start_date.year}.csv'), 
                           parse_dates=['TimeStamp'], index_col='TimeStamp').asfreq('H')

# Read generation profile
gen_profile = pd.read_csv(os.path.join(grid_data_dir, f'gen_profile_{start_date.year}.csv'), 
                           parse_dates=['TimeStamp'], index_col='TimeStamp').asfreq('H')

# Read generator capacity limit profile
genmax_profile = pd.read_csv(os.path.join(grid_data_dir, f'genmax_profile_{start_date.year}.csv'), 
                           parse_dates=['TimeStamp'], index_col='TimeStamp').asfreq('H')

genmin_profile = pd.read_csv(os.path.join(grid_data_dir, f'genmin_profile_{start_date.year}.csv'), 
                           parse_dates=['TimeStamp'], index_col='TimeStamp').asfreq('H')

# Read generator ramp rate profile
genramp30_profile = pd.read_csv(os.path.join(grid_data_dir, f'genramp30_profile_{start_date.year}.csv'), 
                           parse_dates=['TimeStamp'], index_col='TimeStamp').asfreq('H')

# Read generator cost profile (linear)
gencost0_profile = pd.read_csv(os.path.join(grid_data_dir, f'gencost0_profile_{start_date.year}.csv'), 
                           parse_dates=['TimeStamp'], index_col='TimeStamp').asfreq('H')

gencost1_profile = pd.read_csv(os.path.join(grid_data_dir, f'gencost1_profile_{start_date.year}.csv'), 
                           parse_dates=['TimeStamp'], index_col='TimeStamp').asfreq('H')

In [4]:
load_profile.sum(axis=1).sort_values(ascending=False)

### Read variable renewable energy (VRE) data

In [5]:
# Renewable generation time series
current_solar_gen = pd.read_csv(os.path.join(solar_data_dir, f'current_solar_gen_1hr.csv'),
                                parse_dates=['Time'], index_col='Time').asfreq('H')
current_solar_gen.columns = current_solar_gen.columns.astype(int)

future_solar_gen = pd.read_csv(os.path.join(solar_data_dir, f'future_solar_gen_1hr.csv'),
                                parse_dates=['Time'], index_col='Time').asfreq('H')
future_solar_gen.columns = future_solar_gen.columns.astype(int)

onshore_wind_gen = pd.read_csv(os.path.join(onshore_wind_data_dir, f'current_wind_gen_1hr.csv'),
                                parse_dates=['Time'], index_col='Time').asfreq('H')
onshore_wind_gen.columns = onshore_wind_gen.columns.astype(int)

offshore_wind_gen = pd.read_csv(os.path.join(offshore_wind_data_dir, f'power_load_2018.csv'),
                                parse_dates=['timestamp'], index_col='timestamp')
offshore_wind_gen.index = offshore_wind_gen.index.tz_localize('US/Eastern', ambiguous='infer')
offshore_wind_gen.index.freq = 'H'

# Wind farm capacity info
capacity = [816, 1260, 924, 1230]
capacity_nyc, capacity_li = np.sum(capacity[:2]), np.sum(capacity[2:])

# Correct offshore wind generation
offshore_wind_gen['power_nyc'] = np.where(offshore_wind_gen['power_nyc'] > capacity_nyc, capacity_nyc, offshore_wind_gen['power_nyc'])
offshore_wind_gen['power_li'] = np.where(offshore_wind_gen['power_li'] > capacity_li, capacity_li, offshore_wind_gen['power_li'])

In [6]:
# Renewable allocation table
current_solar_2bus = pd.read_csv(os.path.join(solar_data_dir, f'solar_farms_2bus.csv'), index_col='zip_code')
future_solar_2bus = pd.read_csv(os.path.join(solar_data_dir, f'future_solar_farms_2bus.csv'), index_col=0)
onshore_wind_2bus = pd.read_csv(os.path.join(onshore_wind_data_dir, f'onshore_wind_2bus.csv'), index_col=0)

# Aggregate current solar generation
groupby_dict = current_solar_2bus['busIdx'].to_dict()
current_solar_gen_agg = current_solar_gen.groupby(groupby_dict, axis=1).sum()
current_solar_gen_agg = current_solar_gen_agg/1e3 # convert from kW to MW

# Aggregate future solar generation
groupby_dict = future_solar_2bus['busIdx'].to_dict()
future_solar_gen_agg = future_solar_gen.groupby(groupby_dict, axis=1).sum()
future_solar_gen_agg = future_solar_gen_agg/1e3 # convert from kW to MW

# Aggregate onshore wind generation
groupby_dict = onshore_wind_2bus['busIdx'].to_dict()
onshore_wind_gen_agg = onshore_wind_gen.groupby(groupby_dict, axis=1).sum()
onshore_wind_gen_agg = onshore_wind_gen_agg/1e3 # convert from kW to MW

# Aggregate offshore wind generation
offshore_wind_gen_agg = offshore_wind_gen[['power_nyc', 'power_li']].rename(
    columns={'power_nyc': 81, 'power_li': 82})

In [7]:
fig, axs = plt.subplots(4, 1, figsize=(8, 6),
                        sharex='all')
axs[0].plot(current_solar_gen_agg.sum(axis=1), label='Current solar')
axs[1].plot(future_solar_gen_agg.sum(axis=1), label='Future solar')
axs[2].plot(onshore_wind_gen_agg.sum(axis=1), label='Onshore wind')
axs[3].plot(offshore_wind_gen_agg.sum(axis=1), label='Offshore wind')

titles = ['Current solar', 'Future solar', 'Onshore wind', 'Offshore wind']

for i, ax in enumerate(axs):
    ax.set_title(titles[i])

fig.supxlabel('Hour of year')
fig.supylabel('Power (MW)')
fig.suptitle('Generation time series')
fig.tight_layout()

In [8]:
# 18.08% of current solar generation is built before 2018 (base year)
# Scale down current solar generation by 18.08%
pct_current_solar_built = 0.1808
current_solar_gen_agg = current_solar_gen_agg * (1-pct_current_solar_built)

# 90.6% of onshore wind generation is built before 2018 (base year)
# Scale down onshore wind generation by 90.6%
pct_onshore_wind_built = 0.906
onshore_wind_gen_agg = onshore_wind_gen_agg * (1-pct_onshore_wind_built)

In [9]:
vre_profiles = {'CurSol': current_solar_gen_agg,
                'FutSol': future_solar_gen_agg,
                'OnWind': onshore_wind_gen_agg,
                'OffWind': offshore_wind_gen_agg}
vre_prop_list = list()
for key, profile in vre_profiles.items():
    vre_prop_a = pd.DataFrame(data={'VRE_BUS': profile.columns,
                                    'VRE_PMAX': profile.max(axis=0),
                                    'VRE_PMIN': 0,
                                    'VRE_TYPE': 'Solar',
                                    'VRE_NAME': [f'{key}_{col}' for col in profile.columns]})
    vre_prop_list.append(vre_prop_a)
    
# Combine gen_prop tables
vre_prop = pd.concat(vre_prop_list, ignore_index=True)
vre_prop

In [10]:
# Combine genmax tables
genmax_profile_vre = pd.concat([current_solar_gen_agg, future_solar_gen_agg,
                                onshore_wind_gen_agg, offshore_wind_gen_agg], axis=1)
genmax_profile_vre.columns = vre_prop['VRE_NAME']
genmax_profile_vre.index = genmax_profile_vre.index.tz_convert('US/Eastern').tz_localize(None)
genmax_profile_vre

### Read DC line and ESR properties

In [11]:
# Read DC line property file
filename = os.path.join(grid_data_dir, 'dcline_prop.csv')
dcline_prop = pd.read_csv(filename, index_col=0)
dcline_prop

In [12]:
filename = os.path.join(grid_data_dir, 'esr_prop.csv')
esr_prop = pd.read_csv(filename, index_col=0)
esr_prop

## Multi-period OPF

### Without initial conditions

In [13]:
# Only run for one day
d = 240
start_datetime = timestamp_list[d]
end_datetime = start_datetime + timedelta(hours=23)
print(f'Start time: {start_datetime}')
print(f'End time: {end_datetime}')

#### Without ESR

In [14]:
# Create NYGrid object
nygrid_sim_wo_esr = NYGrid(grid_data_dir, 
                            start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                            end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                            dcline_prop=dcline_prop,
                            esr_prop=None,
                            vre_prop=None,
                            verbose=True)

# Set load and generation time series data
nygrid_sim_wo_esr.set_load_sch(load_profile)
nygrid_sim_wo_esr.set_gen_mw_sch(gen_profile)
nygrid_sim_wo_esr.set_gen_max_sch(genmax_profile)
nygrid_sim_wo_esr.set_gen_min_sch(genmin_profile)
nygrid_sim_wo_esr.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_wo_esr.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_wo_esr.relax_external_branch_lim()

# Set generator initial condition
nygrid_sim_wo_esr.set_gen_init_data(gen_init=None)

# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_wo_esr.set_options(options)

# Solve DC OPF
nygrid_sim_wo_esr.solve_dc_opf()

# Get results
results_0_wo_esr = nygrid_sim_wo_esr.get_results_dc_opf()

In [15]:
print("s_ramp_up", results_0_wo_esr['s_ramp_up'].sum())
print("s_ramp_down", results_0_wo_esr['s_ramp_down'].sum())
print("s_over_gen", results_0_wo_esr['s_over_gen'].sum())
print("s_load_shed", results_0_wo_esr['s_load_shed'].sum())
print("s_if_max", results_0_wo_esr['s_if_max'].sum())
print("s_if_min", results_0_wo_esr['s_if_min'].sum())
print("s_br_max", results_0_wo_esr['s_br_max'].sum())
print("s_br_min", results_0_wo_esr['s_br_min'].sum())

#### With ESR

In [16]:
# Create NYGrid object
nygrid_sim_w_esr = NYGrid(grid_data_dir, 
                          start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                          end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                          dcline_prop=dcline_prop,
                          esr_prop=esr_prop,
                          vre_prop=None,
                          verbose=True)

# Set load and generation time series data
nygrid_sim_w_esr.set_load_sch(load_profile)
nygrid_sim_w_esr.set_gen_mw_sch(gen_profile)
nygrid_sim_w_esr.set_gen_max_sch(genmax_profile)
nygrid_sim_w_esr.set_gen_min_sch(genmin_profile)
nygrid_sim_w_esr.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_w_esr.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_w_esr.relax_external_branch_lim()

# Set generator initial condition
nygrid_sim_w_esr.set_gen_init_data(gen_init=None)

# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_w_esr.set_options(options)

# Solve DC OPF
nygrid_sim_w_esr.solve_dc_opf()

# Get results
results_0_w_esr = nygrid_sim_w_esr.get_results_dc_opf()

In [17]:
print("s_ramp_up", results_0_w_esr['s_ramp_up'].sum())
print("s_ramp_down", results_0_w_esr['s_ramp_down'].sum())
print("s_over_gen", results_0_w_esr['s_over_gen'].sum())
print("s_load_shed", results_0_w_esr['s_load_shed'].sum())
print("s_if_max", results_0_w_esr['s_if_max'].sum())
print("s_if_min", results_0_w_esr['s_if_min'].sum())
print("s_br_max", results_0_w_esr['s_br_max'].sum())
print("s_br_min", results_0_w_esr['s_br_min'].sum())

#### With VRE

In [18]:
# Create NYGrid object
nygrid_sim_w_vre = NYGrid(grid_data_dir, 
                          start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                          end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                          dcline_prop=dcline_prop,
                          esr_prop=None,
                          vre_prop=vre_prop,
                          verbose=True)

# Set load and generation time series data
nygrid_sim_w_vre.set_load_sch(load_profile)
nygrid_sim_w_vre.set_gen_mw_sch(gen_profile)
nygrid_sim_w_vre.set_gen_max_sch(genmax_profile)
nygrid_sim_w_vre.set_vre_max_sch(genmax_profile_vre)
nygrid_sim_w_vre.set_gen_min_sch(genmin_profile)
nygrid_sim_w_vre.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_w_vre.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_w_vre.relax_external_branch_lim()

# Set generator initial condition
nygrid_sim_w_vre.set_gen_init_data(gen_init=None)

# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_w_vre.set_options(options)

# Solve DC OPF
nygrid_sim_w_vre.solve_dc_opf()

# Get results
results_0_w_vre = nygrid_sim_w_vre.get_results_dc_opf()

In [19]:
print("s_ramp_up", results_0_w_vre['s_ramp_up'].sum())
print("s_ramp_down", results_0_w_vre['s_ramp_down'].sum())
print("s_over_gen", results_0_w_vre['s_over_gen'].sum())
print("s_load_shed", results_0_w_vre['s_load_shed'].sum())
print("s_if_max", results_0_w_vre['s_if_max'].sum())
print("s_if_min", results_0_w_vre['s_if_min'].sum())
print("s_br_max", results_0_w_vre['s_br_max'].sum())
print("s_br_min", results_0_w_vre['s_br_min'].sum())

#### With VRE and ESR

In [20]:
# Create NYGrid object
nygrid_sim_w_vre_esr = NYGrid(grid_data_dir, 
                          start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                          end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                          dcline_prop=dcline_prop,
                          esr_prop=esr_prop,
                          vre_prop=vre_prop,
                          verbose=True)

# Set load and generation time series data
nygrid_sim_w_vre_esr.set_load_sch(load_profile)
nygrid_sim_w_vre_esr.set_gen_mw_sch(gen_profile)
nygrid_sim_w_vre_esr.set_gen_max_sch(genmax_profile)
nygrid_sim_w_vre_esr.set_vre_max_sch(genmax_profile_vre)
nygrid_sim_w_vre_esr.set_gen_min_sch(genmin_profile)
nygrid_sim_w_vre_esr.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_w_vre_esr.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_w_vre_esr.relax_external_branch_lim()

# Set generator initial condition
nygrid_sim_w_vre_esr.set_gen_init_data(gen_init=None)

# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_w_vre_esr.set_options(options)

# Solve DC OPF
nygrid_sim_w_vre_esr.solve_dc_opf()

# Get results
results_0_w_vre_esr = nygrid_sim_w_vre_esr.get_results_dc_opf()

In [21]:
print("s_ramp_up", results_0_w_vre_esr['s_ramp_up'].sum())
print("s_ramp_down", results_0_w_vre_esr['s_ramp_down'].sum())
print("s_over_gen", results_0_w_vre_esr['s_over_gen'].sum())
print("s_load_shed", results_0_w_vre_esr['s_load_shed'].sum())
print("s_if_max", results_0_w_vre_esr['s_if_max'].sum())
print("s_if_min", results_0_w_vre_esr['s_if_min'].sum())
print("s_br_max", results_0_w_vre_esr['s_br_max'].sum())
print("s_br_min", results_0_w_vre_esr['s_br_min'].sum())

#### Results

In [22]:
ii = 5
print('total', nygrid_sim_w_esr.model.PG[:, nygrid_sim_w_esr.esr_idx[ii]]())
print('discharging', nygrid_sim_w_esr.model.esrPDis[:, ii]())
print('charging', nygrid_sim_w_esr.model.esrPCrg[:, ii]())
print('SOC', nygrid_sim_w_esr.model.esrSOC[:, ii]())

In [23]:
print(f'Total cost: {results_0_wo_esr["total_cost"].sum():.2f}, {results_0_w_esr["total_cost"].sum():.2f}, {results_0_w_vre["total_cost"].sum():.2f}, {results_0_w_vre_esr["total_cost"].sum():.2f}')
print(f'Gen cost: {results_0_wo_esr["gen_cost"].sum():.2f}, {results_0_w_esr["gen_cost"].sum():.2f}, {results_0_w_vre["gen_cost"].sum():.2f}, {results_0_w_vre_esr["gen_cost"].sum():.2f}')
print(f'Over generation penalty: {results_0_wo_esr["over_gen_penalty"].sum():.2f}, {results_0_w_esr["over_gen_penalty"].sum():.2f}, {results_0_w_vre["over_gen_penalty"].sum():.2f}, {results_0_w_vre_esr["over_gen_penalty"].sum():.2f}')
print(f'Load shed penalty: {results_0_wo_esr["load_shed_penalty"].sum():.2f}, {results_0_w_esr["load_shed_penalty"].sum():.2f}, {results_0_w_vre["load_shed_penalty"].sum():.2f}, {results_0_w_vre_esr["load_shed_penalty"].sum():.2f}')
print(f'Ramp up penalty: {results_0_wo_esr["ramp_up_penalty"].sum():.2f}, {results_0_w_esr["ramp_up_penalty"].sum():.2f}, {results_0_w_vre["ramp_up_penalty"].sum():.2f}, {results_0_w_vre_esr["ramp_up_penalty"].sum():.2f}')
print(f'Ramp down penalty: {results_0_wo_esr["ramp_down_penalty"].sum():.2f}, {results_0_w_esr["ramp_down_penalty"].sum():.2f}, {results_0_w_vre["ramp_down_penalty"].sum():.2f}, {results_0_w_vre_esr["ramp_down_penalty"].sum():.2f}')
print(f'Interface max penalty: {results_0_wo_esr["if_max_penalty"].sum():.2f}, {results_0_w_esr["if_max_penalty"].sum():.2f}, {results_0_w_vre["if_max_penalty"].sum():.2f}, {results_0_w_vre_esr["if_max_penalty"].sum():.2f}')
print(f'Interface min penalty: {results_0_wo_esr["if_min_penalty"].sum():.2f}, {results_0_w_esr["if_min_penalty"].sum():.2f}, {results_0_w_vre["if_min_penalty"].sum():.2f}, {results_0_w_vre_esr["if_min_penalty"].sum():.2f}')
print(f'Branch max penalty: {results_0_wo_esr["br_max_penalty"].sum():.2f}, {results_0_w_esr["br_max_penalty"].sum():.2f}, {results_0_w_vre["br_max_penalty"].sum():.2f}, {results_0_w_vre_esr["br_max_penalty"].sum():.2f}')
print(f'Branch min penalty: {results_0_wo_esr["br_min_penalty"].sum():.2f}, {results_0_w_esr["br_min_penalty"].sum():.2f}, {results_0_w_vre["br_min_penalty"].sum():.2f}, {results_0_w_vre_esr["br_min_penalty"].sum():.2f}')

In [24]:
# Create a dict where key is busIdx and value is zoneID
bus_zone_alloc = nygrid_sim_w_esr.grid_data['bus_prop'].set_index('BUS_I').to_dict()['BUS_ZONE']
bus_names = list(bus_zone_alloc.keys())
bus_names_str = [f'Bus{i}' for i in bus_names]

In [25]:
lmp_wo_esr = results_0_wo_esr['LMP']
lmp_wo_esr.columns = bus_names

lmp_w_esr = results_0_w_esr['LMP']
lmp_w_esr.columns = bus_names

lmp_w_vre = results_0_w_vre['LMP']
lmp_w_vre.columns = bus_names

lmp_w_vre_esr = results_0_w_vre_esr['LMP']
lmp_w_vre_esr.columns = bus_names

# Aggregate LMPs by zone
lmp_wo_esr_zone = lmp_wo_esr.groupby(bus_zone_alloc, axis=1).mean()
lmp_w_esr_zone = lmp_w_esr.groupby(bus_zone_alloc, axis=1).mean()
lmp_w_vre_zone = lmp_w_vre.groupby(bus_zone_alloc, axis=1).mean()
lmp_w_vre_esr_zone = lmp_w_vre_esr.groupby(bus_zone_alloc, axis=1).mean()

In [26]:
fig, axs = plt.subplots(7, 2, figsize=(12, 12), sharex='all', sharey='all')
# Plot LMPs w/o ESR
for i, ax in enumerate(axs.flat):
    lmp_wo_esr_zone.iloc[:, i].plot(ax=ax, label='w/o ESR')
    lmp_w_esr_zone.iloc[:, i].plot(ax=ax, label='w/ ESR')
    lmp_w_vre_zone.iloc[:, i].plot(ax=ax, label='w/ VRE')
    lmp_w_vre_esr_zone.iloc[:, i].plot(ax=ax, label='w/ VRE & ESR')
    ax.set_title(lmp_w_esr_zone.columns[i])
    ax.legend()
    
fig.tight_layout()

In [27]:
load_profile.columns = bus_names
load_profile_zone = load_profile.groupby(bus_zone_alloc, axis=1).sum()
load_profile_zone[start_datetime:end_datetime].plot.area(title="Zonal Load")

In [28]:
results_0_wo_esr['LMP'].plot(legend=False, title="Bus LMP (w/o ESR)")
plt.show()

In [29]:
results_0_w_esr['LMP'].plot(legend=False, title="Bus LMP (w/ ESR)")
plt.show()

In [30]:
gen_max = genmax_profile[start_datetime:end_datetime]
gen_pg = results_0_wo_esr['PG'].iloc[:, :303]
gen_pg.columns = gen_max.columns
surplus = gen_max - gen_pg
surplus

### With initial conditions

In [31]:
# Only run for one day
d = d+1
start_datetime = timestamp_list[d]
end_datetime = start_datetime + timedelta(hours=47)
print(f'Start time: {start_datetime}')
print(f'End time: {end_datetime}')

#### Without ESR

In [32]:
# Create NYGrid object
nygrid_sim_1_wo_esr = NYGrid(grid_data_dir, 
                            start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                            end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                            dcline_prop=dcline_prop,
                            esr_prop=None,
                            verbose=True)

# Set load and generation time series data
nygrid_sim_1_wo_esr.set_load_sch(load_profile)
nygrid_sim_1_wo_esr.set_gen_mw_sch(gen_profile)
nygrid_sim_1_wo_esr.set_gen_max_sch(genmax_profile)
nygrid_sim_1_wo_esr.set_gen_min_sch(genmin_profile)
nygrid_sim_1_wo_esr.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_1_wo_esr.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_1_wo_esr.relax_external_branch_lim()

# Set generator initial condition
last_gen = results_0_wo_esr['PG'].loc[start_datetime].to_numpy().squeeze()
nygrid_sim_1_wo_esr.set_gen_init_data(gen_init=last_gen)

# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_1_wo_esr.set_options(options)

# Solve DC OPF
nygrid_sim_1_wo_esr.solve_dc_opf()

# Get results
results_1_wo_esr = nygrid_sim_1_wo_esr.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_1_wo_esr['s_ramp_up'].sum())
print("s_ramp_down", results_1_wo_esr['s_ramp_down'].sum())
print("s_over_gen", results_1_wo_esr['s_over_gen'].sum())
print("s_load_shed", results_1_wo_esr['s_load_shed'].sum())
print("s_if_max", results_1_wo_esr['s_if_max'].sum())
print("s_if_min", results_1_wo_esr['s_if_min'].sum())
print("s_br_max", results_1_wo_esr['s_br_max'].sum())
print("s_br_min", results_1_wo_esr['s_br_min'].sum())

#### With ESR

In [ ]:
# Create NYGrid object
nygrid_sim_1_w_esr = NYGrid(grid_data_dir, 
                            start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                            end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                            dcline_prop=dcline_prop,
                            esr_prop=esr_prop,
                            verbose=True)

# Set load and generation time series data
nygrid_sim_1_w_esr.set_load_sch(load_profile)
nygrid_sim_1_w_esr.set_gen_mw_sch(gen_profile)
nygrid_sim_1_w_esr.set_gen_max_sch(genmax_profile)
nygrid_sim_1_w_esr.set_gen_min_sch(genmin_profile)
nygrid_sim_1_w_esr.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_1_w_esr.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_1_w_esr.relax_external_branch_lim()

# Set generator initial condition
last_gen = results_0_w_esr['PG'].loc[start_datetime].to_numpy().squeeze()
nygrid_sim_1_w_esr.set_gen_init_data(gen_init=last_gen)

# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_1_w_esr.set_options(options)

# Solve DC OPF
nygrid_sim_1_w_esr.solve_dc_opf()

# Get results
results_1_w_esr = nygrid_sim_1_w_esr.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_1_w_esr['s_ramp_up'].sum())
print("s_ramp_down", results_1_w_esr['s_ramp_down'].sum())
print("s_over_gen", results_1_w_esr['s_over_gen'].sum())
print("s_load_shed", results_1_w_esr['s_load_shed'].sum())
print("s_if_max", results_1_w_esr['s_if_max'].sum())
print("s_if_min", results_1_w_esr['s_if_min'].sum())
print("s_br_max", results_1_w_esr['s_br_max'].sum())
print("s_br_min", results_1_w_esr['s_br_min'].sum())

#### With VRE

In [ ]:
# Create NYGrid object
nygrid_sim_1_w_vre = NYGrid(grid_data_dir, 
                            start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                            end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                            dcline_prop=dcline_prop,
                            esr_prop=None,
                            vre_prop=vre_prop,
                            verbose=True)

# Set load and generation time series data
nygrid_sim_1_w_vre.set_load_sch(load_profile)
nygrid_sim_1_w_vre.set_gen_mw_sch(gen_profile)
nygrid_sim_1_w_vre.set_gen_max_sch(genmax_profile)
nygrid_sim_1_w_vre.set_vre_max_sch(genmax_profile_vre)
nygrid_sim_1_w_vre.set_gen_min_sch(genmin_profile)
nygrid_sim_1_w_vre.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_1_w_vre.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_1_w_vre.relax_external_branch_lim()

# Set generator initial condition
last_gen = results_0_w_vre['PG'].loc[start_datetime].to_numpy().squeeze()
nygrid_sim_1_w_vre.set_gen_init_data(gen_init=last_gen)

# Set options
# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_1_w_vre.set_options(options)

# Solve DC OPF
nygrid_sim_1_w_vre.solve_dc_opf()

# Get results
results_1_w_vre = nygrid_sim_1_w_vre.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_1_w_vre['s_ramp_up'].sum())
print("s_ramp_down", results_1_w_vre['s_ramp_down'].sum())
print("s_over_gen", results_1_w_vre['s_over_gen'].sum())
print("s_load_shed", results_1_w_vre['s_load_shed'].sum())
print("s_if_max", results_1_w_vre['s_if_max'].sum())
print("s_if_min", results_1_w_vre['s_if_min'].sum())
print("s_br_max", results_1_w_vre['s_br_max'].sum())
print("s_br_min", results_1_w_vre['s_br_min'].sum())

#### With VRE and ESR

In [ ]:
# Create NYGrid object
nygrid_sim_1_w_vre_esr = NYGrid(grid_data_dir, 
                            start_datetime=start_datetime.strftime('%m-%d-%Y %H'), 
                            end_datetime=end_datetime.strftime('%m-%d-%Y %H'),
                            dcline_prop=dcline_prop,
                            esr_prop=esr_prop,
                            vre_prop=vre_prop,
                            verbose=True)

# Set load and generation time series data
nygrid_sim_1_w_vre_esr.set_load_sch(load_profile)
nygrid_sim_1_w_vre_esr.set_gen_mw_sch(gen_profile)
nygrid_sim_1_w_vre_esr.set_gen_max_sch(genmax_profile)
nygrid_sim_1_w_vre_esr.set_vre_max_sch(genmax_profile_vre)
nygrid_sim_1_w_vre_esr.set_gen_min_sch(genmin_profile)
nygrid_sim_1_w_vre_esr.set_gen_ramp_sch(genramp30_profile)
nygrid_sim_1_w_vre_esr.set_gen_cost_sch(gencost0_profile, gencost1_profile)

# Relax branch flow limits
nygrid_sim_1_w_vre_esr.relax_external_branch_lim()

# Set generator initial condition
last_gen = results_0_w_vre_esr['PG'].loc[start_datetime].to_numpy().squeeze()
nygrid_sim_1_w_vre_esr.set_gen_init_data(gen_init=last_gen)

# Set options
options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 20_000,
    'PenaltyForBranchMwViolation': 10_000,
    'PenaltyForInterfaceMWViolation': 10_000
}

nygrid_sim_1_w_vre_esr.set_options(options)

# Solve DC OPF
nygrid_sim_1_w_vre_esr.solve_dc_opf()

# Get results
results_1_w_vre_esr = nygrid_sim_1_w_vre_esr.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_1_w_vre_esr['s_ramp_up'].sum())
print("s_ramp_down", results_1_w_vre_esr['s_ramp_down'].sum())
print("s_over_gen", results_1_w_vre_esr['s_over_gen'].sum())
print("s_load_shed", results_1_w_vre_esr['s_load_shed'].sum())
print("s_if_max", results_1_w_vre_esr['s_if_max'].sum())
print("s_if_min", results_1_w_vre_esr['s_if_min'].sum())
print("s_br_max", results_1_w_vre_esr['s_br_max'].sum())
print("s_br_min", results_1_w_vre_esr['s_br_min'].sum())

## Process the results

In [ ]:
# Read thermal generator info table
filename = os.path.join(data_dir, 'genInfo.csv')
gen_info = pd.read_csv(filename)
num_thermal = gen_info.shape[0]
gen_rename = {gen_info.index[i]: gen_info.NYISOName[i] for i in range(num_thermal)}
gen_info

In [ ]:
# Format results
results_pg_0_wo_esr = results_0_wo_esr['PG']
thermal_pg_0_wo_esr = results_pg_0_wo_esr.iloc[:, :num_thermal]
thermal_pg_0_wo_esr = thermal_pg_0_wo_esr.rename(columns=gen_rename)
print(thermal_pg_0_wo_esr.head())

results_pg_0_w_esr = results_0_w_esr['PG']
thermal_pg_0_w_esr = results_pg_0_w_esr.iloc[:, :num_thermal]
thermal_pg_0_w_esr = thermal_pg_0_w_esr.rename(columns=gen_rename)
print(thermal_pg_0_w_esr.head())

results_pg_1_wo_esr = results_1_wo_esr['PG']
thermal_pg_1_wo_esr = results_pg_1_wo_esr.iloc[:, :num_thermal]
thermal_pg_1_wo_esr = thermal_pg_1_wo_esr.rename(columns=gen_rename)
print(thermal_pg_1_wo_esr.head())

results_pg_1_w_esr = results_1_w_esr['PG']
thermal_pg_1_w_esr = results_pg_1_w_esr.iloc[:, :num_thermal]
thermal_pg_1_w_esr = thermal_pg_1_w_esr.rename(columns=gen_rename)
print(thermal_pg_1_w_esr.head())

In [ ]:
def plot_gen(thermal_pg, genhist, genmax, genmin, idx, title=None, ax=None):

    ax.plot(thermal_pg.index, thermal_pg.iloc[:, idx], marker='*', label='OPF')
    ax.plot(thermal_pg.index, genhist[start_datetime:end_datetime].iloc[:,idx], marker='o', label='historical')
    ax.plot(thermal_pg.index, genmax[start_datetime:end_datetime].iloc[:,idx], linestyle='--', label='max')
    ax.plot(thermal_pg.index, genmin[start_datetime:end_datetime].iloc[:,idx], linestyle='--', label='min')
    ax.legend()
    ax.set_title(title)

    return ax

In [ ]:
gen_profile[start_datetime:end_datetime]

In [ ]:
ii = 3
print(gen_info.iloc[ii, :])
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs[0, 0] = plot_gen(thermal_pg_0_wo_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[0, 0])
axs[0, 1] = plot_gen(thermal_pg_0_w_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[0, 1])
axs[1, 0] = plot_gen(thermal_pg_1_wo_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[1, 0])
axs[1, 1] = plot_gen(thermal_pg_1_w_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[1, 1])
fig.tight_layout()

In [ ]:
ii = 9
print(gen_info.iloc[ii, :])
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs[0, 0] = plot_gen(thermal_pg_0_wo_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[0, 0])
axs[0, 1] = plot_gen(thermal_pg_0_w_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[0, 1])
axs[1, 0] = plot_gen(thermal_pg_1_wo_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[1, 0])
axs[1, 1] = plot_gen(thermal_pg_1_w_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[1, 1])
fig.tight_layout()

In [ ]:
ii = 83
print(gen_info.iloc[ii, :])
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs[0, 0] = plot_gen(thermal_pg_0_wo_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[0, 0])
axs[0, 1] = plot_gen(thermal_pg_0_w_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[0, 1])
axs[1, 0] = plot_gen(thermal_pg_1_wo_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[1, 0])
axs[1, 1] = plot_gen(thermal_pg_1_w_esr, gen_profile, genmax_profile, genmin_profile, ii, title=gen_info.NYISOName[ii], ax=axs[1, 1])
plt.show()